In [ ]:
import os

%matplotlib inline
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(11) #random number
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
import itertools
import cv2
import skimage.color
import skimage.filters
from skimage import io, exposure, morphology, filters, color, segmentation, feature, measure, img_as_float, img_as_ubyte
from skimage.segmentation import chan_vese
from skimage.measure import label, regionprops
import math
from PIL import Image, ImageChops, ImageStat
!pip install extcolors
import extcolors

In [ ]:
import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Dropout, LSTM, Embedding, Reshape, GlobalAveragePooling2D
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from keras import backend as K
from tensorflow.keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
# add other models as required
from keras import backend as K 

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_fscore_support as score, precision_score, recall_score, f1_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import RFECV

from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

import pickle
from tensorflow.keras.utils import plot_model
from keras.models import load_model

**Step 2 : Loading pictures and making Dictionary of images and labels**

In this step I load in the pictures and turn them into numpy arrays using their RGB values. As the pictures have already been resized to 224x224, there's no need to resize them. As the pictures do not have any labels, these need to be created. Finally, the pictures are added together to a big training set and shuffeled.

In [ ]:
folder_benign_train = '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign'
folder_malignant_train = '/kaggle/input/skin-cancer-malignant-vs-benign/train/malignant'

folder_benign_test = '/kaggle/input/skin-cancer-malignant-vs-benign/test/benign'
folder_malignant_test = '/kaggle/input/skin-cancer-malignant-vs-benign/test/malignant'

read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))

In [ ]:
# Load in training pictures 
ims_benign = [read(os.path.join(folder_benign_train, filename)) for filename in os.listdir(folder_benign_train)]
X_benign = np.array(ims_benign, dtype='uint8')
ims_malignant = [read(os.path.join(folder_malignant_train, filename)) for filename in os.listdir(folder_malignant_train)]
X_malignant = np.array(ims_malignant, dtype='uint8')

In [ ]:
print(X_benign.shape)
print(X_malignant.shape)

In [ ]:
# Load in testing pictures
ims_benign = [read(os.path.join(folder_benign_test, filename)) for filename in os.listdir(folder_benign_test)]
X_benign_test = np.array(ims_benign, dtype='uint8')
ims_malignant = [read(os.path.join(folder_malignant_test, filename)) for filename in os.listdir(folder_malignant_test)]
X_malignant_test = np.array(ims_malignant, dtype='uint8')

In [ ]:
# Create labels
y_benign = np.zeros(X_benign.shape[0])
y_malignant = np.ones(X_malignant.shape[0])

y_benign_test = np.zeros(X_benign_test.shape[0])
y_malignant_test = np.ones(X_malignant_test.shape[0])

In [ ]:
# Merge data 
X_train = np.concatenate((X_benign, X_malignant), axis = 0)
y_train = np.concatenate((y_benign, y_malignant), axis = 0)

X_test = np.concatenate((X_benign_test, X_malignant_test), axis = 0)
y_test = np.concatenate((y_benign_test, y_malignant_test), axis = 0)

In [ ]:
print(y_train)

In [ ]:
# Shuffle data
# dataset already shuffled
'''
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
y_train = y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
y_test = y_test[s]
'''

**imshow_all()** allows to plot a series of images side-by-side using matplotlib.

In [ ]:
def imshow_all(*images, **kwargs):
    """
    Plot a series of images side-by-side.

    Convert all images to float so that images have a common intensity range.

    Parameters
    ----------
    limits : str
        Control the intensity limits. By default, 'image' is used set the
        min/max intensities to the min/max of all images. Setting `limits` to
        'dtype' can also be used if you want to preserve the image exposure.
    titles : list of str
        Titles for subplots. If the length of titles is less than the number
        of images, empty strings are appended.
    kwargs : dict
        Additional keyword-arguments passed to `imshow`.
    """
    images = [img_as_float(img) for img in images]

    titles = kwargs.pop('titles', [])
    if len(titles) != len(images):
        titles = list(titles) + [''] * (len(images) - len(titles))

    limits = kwargs.pop('limits', 'image')
    if limits == 'image':
        kwargs.setdefault('vmin', min(img.min() for img in images))
        kwargs.setdefault('vmax', max(img.max() for img in images))
    elif limits == 'dtype':
        vmin, vmax = dtype_limits(images[0])
        kwargs.setdefault('vmin', vmin)
        kwargs.setdefault('vmax', vmax)

    nrows, ncols = kwargs.get('shape', (1, len(images)))
    
    axes_off = kwargs.pop('axes_off', False)

    size = nrows * kwargs.pop('size', 5)
    width = size * len(images)
    if nrows > 1:
        width /= nrows * 1.33
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(width, size))
    for ax, img, label in zip(axes.ravel(), images, titles):
        ax.imshow(img, **kwargs)
        ax.set_title(label)
        ax.grid(False)
        if axes_off:
            ax.set_axis_off()
    fig.savefig('images.png', dpi = 300)

In [ ]:
def dullrazor(img, showimgs, lowbound=15, filterstruc=3, inpaintmat=3):
    
    #grayscale
    imgtmp1 = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    #applying a blackhat
    filterSize =(filterstruc, filterstruc)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
    imgtmp2 = cv2.morphologyEx(imgtmp1, cv2.MORPH_BLACKHAT, kernel)

    #0=skin and 255=hair
    ret, mask = cv2.threshold(imgtmp2, lowbound, 255, cv2.THRESH_BINARY)
    
    #inpainting
    img_final = cv2.inpaint(img, mask, inpaintmat ,cv2.INPAINT_TELEA)
    
    
    if showimgs:
        fig=plt.figure(figsize=(24, 16))
        columns = 5
        rows = 1
        
        fig.add_subplot(rows, columns, 1)
        plt.imshow(img, interpolation='nearest')
        
        fig.add_subplot(rows, columns, 2)
        plt.imshow(imgtmp1, cmap="gray")
        
        fig.add_subplot(rows, columns, 3)
        plt.imshow(imgtmp2, cmap='gray')
        
        fig.add_subplot(rows, columns, 4)
        plt.imshow(mask, cmap='gray')
        
        fig.add_subplot(rows, columns, 5)
        plt.imshow(img_final)
        print("___________________")


    return img_final

In [ ]:
img_test = read(os.path.join("/kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/", "1012.jpg"))
img_test_array = np.array(img_test, dtype='uint8')

In [ ]:
plt.imshow(img_test_array, interpolation='nearest')
plt.savefig('sample.png', dpi = 300)
plt.show()

In [ ]:
img_p1 = dullrazor(img_test_array,showimgs=True)
plt.imshow(img_p1, interpolation='nearest')
plt.savefig('dullrazor.png', dpi = 300)
plt.show()

In [ ]:
fig=plt.figure(figsize=(48, 32))
columns = 2
rows = 1

img_p2 = cv2.fastNlMeansDenoisingColored(img_p1,None,2,2,7,21)

fig.add_subplot(rows, columns, 1)
plt.imshow(img_p1, interpolation='nearest')

fig.add_subplot(rows, columns, 2)
plt.imshow(img_p2, interpolation='nearest')
fig.savefig('fastNl.png', dpi = 300)

In [ ]:
import skimage.color
import skimage.filters
gray_image = skimage.color.rgb2gray(img_p2)
plt.imshow(gray_image, cmap="gray")
plt.savefig('bg.png', dpi = 300)

# show the histogram of the gray-scaled image
histogram, bin_edges = np.histogram(gray_image, bins=256, range=(0.0, 1.0))
fig, ax = plt.subplots()
plt.plot(bin_edges[0:-1], histogram)
plt.title("Graylevel histogram")
plt.xlabel("gray value")
plt.ylabel("pixel count")
plt.xlim(0, 1.0)
plt.savefig('histo.png', dpi = 300)

The histogram has a peaks starting from 0.2 to a little more than 0.8, with a significant peak around 0.6. Thus, this image is a good candidate for thresholding with Otsu’s method. The outcome is that Otsu’s method finds a threshold value between the two peaks of a grayscale histogram.

The skimage.filters.threshold_otsu() function can be used to determine the threshold automatically via Otsu’s method. Then numpy comparison operators can be used to apply it as before. 

In [ ]:
# perform automatic thresholding
t = skimage.filters.threshold_otsu(gray_image)
print("Found automatic threshold t = {}.".format(t))

For this root image and a Gaussian blur with the chosen sigma of 1.0, the computed threshold value is 0.58. No we can create a binary mask with the comparison operator <. As we have seen before, pixels above the threshold value will be turned on, those below the threshold will be turned off.



In [ ]:
# create a binary mask with the threshold found by Otsu's method
binary_mask = gray_image < t

fig, ax = plt.subplots()
plt.imshow(binary_mask, cmap="gray")
plt.savefig('mask.png', dpi = 300)

Finally, we use the mask to select the foreground:

In [ ]:
# apply the binary mask to select the foreground
selection = img_p2.copy()
selection[~binary_mask] = 0

fig, ax = plt.subplots()
plt.imshow(selection)
plt.savefig('mask_2.png', dpi = 300)

In [ ]:
#image = img_as_float(data.camera())
# Feel free to play around with the parameters to see how they impact the result
image = skimage.color.rgb2gray(selection)
cv = chan_vese(image, mu=0.5, lambda1=1, lambda2=1, tol=1e-3, max_num_iter=200,
               dt=0.5, init_level_set="checkerboard", extended_output=True)

fig, axes = plt.subplots(2, 2, figsize=(8, 8))
ax = axes.flatten()

ax[0].imshow(image, cmap="gray")
ax[0].set_axis_off()
ax[0].set_title("Original Image", fontsize=12)

ax[1].imshow(cv[0], cmap="gray")
ax[1].set_axis_off()
title = "Chan-Vese segmentation - {} iterations".format(len(cv[2]))
ax[1].set_title(title, fontsize=12)

ax[2].imshow(cv[1], cmap="gray")
ax[2].set_axis_off()
ax[2].set_title("Final Level Set", fontsize=12)

ax[3].plot(cv[2])
ax[3].set_title("Evolution of energy over iterations", fontsize=12)

fig.tight_layout()
fig.savefig('chan_vese.png', dpi = 300)
plt.show()

In [ ]:
def preprocessing(image): # to create the black and white contrast mask used to calculate ABD values
    image = dullrazor(image, showimgs=False)
    image = cv2.fastNlMeansDenoisingColored(image,None,2,2,7,21)
    image_gray = skimage.color.rgb2gray(image)
    t = skimage.filters.threshold_otsu(image_gray)
    binary_mask = image_gray < t
    selection = image.copy()
    selection[~binary_mask] = 0
    image = skimage.color.rgb2gray(selection)
    cv = chan_vese(image, mu=0.5, lambda1=1, lambda2=1, tol=1e-3, max_num_iter=100,
               dt=0.5, init_level_set="checkerboard", extended_output=True)
    y = 1*cv[0]
    return y

In [ ]:
def partial_preprocessing(image): # to overlay the black mask on the image to get the color varegation value
    image = dullrazor(image, showimgs=False)
    image = cv2.fastNlMeansDenoisingColored(image,None,2,2,7,21)
    image_gray = skimage.color.rgb2gray(image)
    t = skimage.filters.threshold_otsu(image_gray)
    binary_mask = image_gray < t
    selection = image.copy()
    selection[~binary_mask] = 0

    return selection

In [ ]:
img_test = read(os.path.join("/kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/", "1012.jpg"))
img_test_array = np.array(img_test, dtype='uint8')
plt.imshow(preprocessing(img_test_array), cmap="gray")
plt.savefig('finalcontour.png', dpi = 300)

In [ ]:
label_img = label(preprocessing(img_test_array), connectivity=preprocessing(img_test_array).ndim)
props = regionprops(label_img)
centroid = props[0].centroid
#print(centroid)

fig=plt.figure(figsize=(24, 16))
columns = 5
rows = 1

# segmented image with centroid
fig.add_subplot(rows, columns, 1)
plt.plot(centroid[0],centroid[1], marker='.', color="blue")
plt.imshow(preprocessing(img_test_array),cmap = "gray")


# rotation about centroid by orientation angle
angle = props[0].orientation * (180/math.pi)
M = cv2.getRotationMatrix2D((centroid[0],centroid[1]), angle, 1.0)
rotated_image = cv2.warpAffine(preprocessing(img_test_array).astype(np.uint8), M, (224, 224))
fig.add_subplot(rows, columns, 2)
plt.plot(centroid[0],centroid[1], marker='.', color="blue")
plt.imshow(rotated_image,cmap = "gray")


# transpose to centre
hh, ww = rotated_image.shape
# get contours (presumably just one around the nonzero pixels) 
contours = cv2.findContours(rotated_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]
for cntr in contours:
    x,y,w,h = cv2.boundingRect(cntr)
    
# recenter
startx = (ww - w)//2
starty = (hh - h)//2
result = np.zeros_like(rotated_image)
result[starty:starty+h,startx:startx+w] = rotated_image[y:y+h,x:x+w]
# view result
fig.add_subplot(rows, columns, 3)
plt.plot(112,112, marker='.', color="blue")
plt.imshow(result,cmap = "gray")


#flipped horixontally along x-axis
flipped_x = cv2.flip(result, 0) # 0 for x-axis
fig.add_subplot(rows, columns, 4)
plt.plot(112,112, marker='.', color="blue")
plt.imshow(flipped_x,cmap = "gray")


#flipped horixontally along y-axis
flipped_y = cv2.flip(result, 1) # 1 for y-axis
fig.add_subplot(rows, columns, 5)
plt.plot(112,112, marker='.', color="blue")
plt.imshow(flipped_y,cmap = "gray")

In [ ]:
# ABCD rule
fig=plt.figure(figsize=(12,8))
columns = 2
rows = 1

# XOR with Numpy
dLx = np.bitwise_xor(result,flipped_x).astype(np.uint8)
dLy = np.bitwise_xor(result,flipped_y).astype(np.uint8)

fig.add_subplot(rows, columns, 1)
plt.imshow(dLx,cmap = "gray")
plt.savefig('dLx.png', dpi = 300)

fig.add_subplot(rows, columns, 2)
plt.imshow(dLy,cmap = "gray")
plt.savefig('dLy.png', dpi = 300)


# Color variegation:
im = Image.fromarray(selection.astype('uint8'), 'RGB')
stat = ImageStat.Stat(im)
C_r = stat.stddev[0] / stat.extrema[0][1]
C_g = stat.stddev[1] / stat.extrema[1][1]
C_b = stat.stddev[2] / stat.extrema[2][1]

# Texture: Gray Level Co-occurrence Matrix (GLCM)
gray_img = cv2.cvtColor(img_test_array, cv2.COLOR_BGR2GRAY)
glcm = feature.graycomatrix(image=img_as_ubyte(gray_img), distances=[1],angles=[0, np.pi/4, np.pi/2, np.pi * 3/2],symmetric=True, normed=True)
correlation = np.mean(feature.graycoprops(glcm, prop='correlation'))
homogeneity = np.mean(feature.graycoprops(glcm, prop='homogeneity'))
energy = np.mean(feature.graycoprops(glcm, prop='energy'))
contrast = np.mean(feature.graycoprops(glcm, prop='contrast'))


# property calculations
r = regionprops(result)[0]
a1 = regionprops(dLx)[0].area/r.area # Asymmetry score across x-axis
a2 = regionprops(dLy)[0].area/r.area # Asymmetry score across y-axis
AI = (a1+a2)/2 # Asymmetricity Index

b1 = r.area/r.perimeter # Area to perimeter ratio
b2 = (4*math.pi*r.area)/(r.perimeter*r.perimeter) # Compactness index
b3 = r.perimeter*r.area # Perimeter multiplied by area

D1 = math.sqrt((4*r.area)/math.pi)
D2 = (r.axis_major_length+r.axis_minor_length)/2
d1 = ((D1+D2)/2)*0.265 # Average diameter of the lesion (1px = 0.265mm)
d2 = r.axis_major_length-r.axis_minor_length # Difference between principal axes lengths


# print property values
print("Perimeter {:.3f}: ".format(r.perimeter))
print("Area {:.3f}:".format(r.area))
print("A1 (Asymmetry score across x-axis) {:.3f}: ".format(a1))
print("A2 (Asymmetry score across y-axis) {:.3f}: ".format(a2))
print("AI (Asymmetricity Index) {:.3f}: ".format(AI))

print("B1 (Area to perimeter ratio) {:.3f}: ".format(b1))
print("B2 (Compactness index) {:.3f}:".format(b2))
print("B3 (Perimeter multiplied by area) {:.3f}:".format(b3))

print("D1 (Average diameter of the lesion) {:.3f}:".format(d1))
print("D2 (Difference between principal axes lengths) {:.3f}:".format(d2))

print('Red Std Deviation: {:.3f}'.format(C_r))
print('Green Std Deviation: {:.3f}'.format(C_g))
print('Blue Std Deviation: {:.3f}'.format(C_b))

print('Correlation: {:.3f}'.format(correlation))
print('Homogeneity: {:.3f}'.format(homogeneity))
print('Energy: {:.3f}'.format(energy))
print('Contrast: {:.3f}'.format(contrast))

In [ ]:
# number of distict color classes whose presence in the image>5%

PIL_image = Image.fromarray(selection.astype('uint8'), 'RGB')
colors_x = extcolors.extract_from_image(PIL_image, tolerance = 4, limit = 50)

def hextriplet(colortuple):
    return '#' + ''.join(f'{i:02X}' for i in colortuple)

def color_to_df(input):
    colors_pre_list = str(input).replace('([(','').split(', (')[0:-1]
    df_rgb = [i.split('), ')[0] + ')' for i in colors_pre_list]
    df_percent = [i.split('), ')[1].replace(')','') for i in colors_pre_list]
    #print(type(df_rgb[0]))
    #convert RGB to HEX code
    
    df_color_up = [hextriplet(tuple(map(int, ((test_str.replace('(','')).replace(')','')).split(', ')))) for test_str in df_rgb]
    
    df = pd.DataFrame(zip(df_color_up,df_rgb,df_percent), columns = ['c_code','rgb','occurence'])
    return df

df_color = color_to_df(colors_x)

list_color = list(df_color['c_code'])
list_occurrence = [int(i) for i in list(df_color['occurence'])]
list_rgb = [tuple(map(int, ((test_str.replace('(','')).replace(')','')).split(', '))) for test_str in list(df_color['rgb'])] 
list_percent = [round((p*100)/sum(list_occurrence),3) for p in list_occurrence]

color = [0]*6
for i in range(0,len(list_rgb)):
    r = list_rgb[i][0]
    g = list_rgb[i][1]
    b = list_rgb[i][2]
    p = list_percent[i]
    
    if r>=(0.8*255) and g>=(0.8*255) and b>=(0.8*255):
        color[0]+=p #white
    elif r>=(0.588*255) and g<(0.2*255) and b<(0.2*255):
        color[1]+=p #red
    elif (r>=(0.588*255) and r<=(0.94*255)) and (g>(0.2*255) and g<=(0.588*255)) and (b>0 and b<(0.392*255)):
        color[2]+=p #lightbrown
    elif (r>(0.243*255) and r<(0.56*255)) and g<(0.392*255) and (b>0 and b<(0.392*255)):
        color[3]+=p #darkbrown
    elif r<=(0.588*255) and (g>=(0.392*255) and g<=(0.588*255)) and (b>=(0.490*255) and b<=(0.588*255)):
        color[4]+=p #bluegray
    elif r<=(0.243*255) and g<=(0.243*255) and b<=(0.243*255):
        color[5]+=p #black
    
# using list comprehension
# to get numbers > 5%
count = len([i for i in color if i > 5])    
print("Number of distinct color classes :", count)

In [ ]:
# Total dermoscopic score ≥ 5.45 is highly suggestive of melanoma
#Total dermoscopic score = 1.3 (A score) + 0.1 (B score) + 0.5 (C score) + 0.5 (D score)
TDS_old1 = (1.3*AI)+(0.1*b2)+(0.5*count)+(0.5*d1)
TDS_old2 = (1.3*AI)+(0.1*(1/b2))+(0.5*count)+(0.5*d1)
print(TDS_old1)
print(TDS_old2)

#New TDS = (1.3*AsymmetryIndex)+(0.1*BorderIrregularity)
#+(0.5*Black)+(0.5*White)+(0.5*Red)+(0.5*Blue)+(0.3*DarkBrown)+(0.4*LightBrown)+(0.5*Diameter)
TDS_new1 = (1.3*AI)+(0.1*b2)+(0.5*color[0])+(0.5*color[1])+(0.4*color[2])+(0.3*color[3])+(0.5*color[4])+(0.5*color[5])+(0.5*d1)
TDS_new2 = (1.3*AI)+(0.1*(1/b2))+(0.5*color[0])+(0.5*color[1])+(0.4*color[2])+(0.3*color[3])+(0.5*color[4])+(0.5*color[5])+(0.5*d1)
print(TDS_new1)
print(TDS_new2)


In [ ]:
#feature extration (attempt 2)

lesion_region = regionprops(preprocessing(img_test_array))[0]
area_total = lesion_region.area
img_mask = lesion_region.image

horizontal_flip = np.fliplr(img_mask)
diff_horizontal = img_mask * ~horizontal_flip

vertical_flip = np.flipud(img_mask)
diff_vertical = img_mask * ~vertical_flip
diff_horizontal_area = np.count_nonzero(diff_horizontal)
diff_vertical_area = np.count_nonzero(diff_vertical)
a1 = diff_horizontal_area / area_total
a2 = diff_vertical_area / area_total
asymm_idx = 0.5 * (a1 + a2)
ecc = lesion_region.eccentricity

print('Primeter: {:.3f}'.format(lesion_region.perimeter))
print('Area: {:.3f}'.format(lesion_region.area))

print('-- ASYMMETRY --')
print('Diff area horizontal: {:.3f}'.format(np.count_nonzero(diff_horizontal)))
print('Diff area vertical: {:.3f}'.format(np.count_nonzero(diff_vertical)))
print("A1 (Asymmetry score across x-axis) {:.3f}: ".format(a1))
print("A2 (Asymmetry score across y-axis) {:.3f}: ".format(a2))
print('Asymmetric Index: {:.3f}'.format(asymm_idx))
print('Eccentricity: {:.3f}'.format(ecc))
# print('Minor-Major axis ratio: {}'.format(mmr))
        
imshow_all(img_mask, horizontal_flip, diff_horizontal,titles=['image mask', 'horizontal flip', 'difference'], size=4, cmap='gray')
plt.savefig('transforms1.png', dpi = 300)
imshow_all(img_mask, vertical_flip, diff_vertical,titles=['image mask', 'vertical flip', 'difference'], size=4, cmap='gray')
plt.savefig('transforms2.png', dpi = 300)
plt.show();

# Border Irregularity: Compactness index
compact_index = (4 * np.pi * area_total)/(lesion_region.perimeter ** 2) #formula confusion(cleck later)
b1 =  lesion_region.area/lesion_region.perimeter # area to perimeter ratio
print('\n-- BORDER IRREGULARITY --')

print('Compact Index: {:.3f}'.format(compact_index))

# Diameter
eq_diameter = lesion_region.equivalent_diameter
D1 = math.sqrt((4*lesion_region.area)/math.pi)
D2 = (lesion_region.axis_major_length+lesion_region.axis_minor_length)/2
d1 = ((D1+D2)/2)*0.265 # Average diameter of the lesion (1px = 0.265mm)
d2 = lesion_region.axis_major_length-lesion_region.axis_minor_length # Difference between principal axes lengths
print('\n-- DIAMETER --')
#print('Equivalent diameter: {:.3f}'.format(eq_diameter))
# optionally convert the diameter in mm, knowing that 1 px = 0.265 mm:
# 1 px : 0.265 mm = diam_px : diam_mm -> diam_mm = diam_px * 0.265
print('Diameter (mm): {:.3f}'.format(eq_diameter * 0.265))
print("D1 (Average diameter of the lesion) {:.3f}:".format(d1))
print("D2 (Difference between principal axes lengths) {:.3f}:".format(d2))

# Color variegation:
im = Image.fromarray(selection.astype('uint8'), 'RGB')
stat = ImageStat.Stat(im)
C_r = stat.stddev[0] / stat.extrema[0][1]
C_g = stat.stddev[1] / stat.extrema[1][1]
C_b = stat.stddev[2] / stat.extrema[2][1]
print('\n-- COLOR VARIEGATION --')
print('Red Std Deviation: {:.3f}'.format(C_r))
print('Green Std Deviation: {:.3f}'.format(C_g))
print('Blue Std Deviation: {:.3f}'.format(C_b))

# Texture: Gray Level Co-occurrence Matrix (GLCM)
gray_image = cv2.cvtColor(img_test_array, cv2.COLOR_BGR2GRAY)
glcm = feature.graycomatrix(image=img_as_ubyte(gray_img), distances=[1],angles=[0, np.pi/4, np.pi/2, np.pi * 3/2],symmetric=True, normed=True)
correlation = np.mean(feature.graycoprops(glcm, prop='correlation'))
homogeneity = np.mean(feature.graycoprops(glcm, prop='homogeneity'))
energy = np.mean(feature.graycoprops(glcm, prop='energy'))
contrast = np.mean(feature.graycoprops(glcm, prop='contrast'))
print('\n-- TEXTURE --')
print('Correlation: {:.3f}'.format(correlation))
print('Homogeneity: {:.3f}'.format(homogeneity))
print('Energy: {:.3f}'.format(energy))
print('Contrast: {:.3f}'.format(contrast))


In [ ]:
# using feature selection process 2
def feature_extraction(img_test_array):
    properties = list()
    lesion_region = regionprops(preprocessing(img_test_array))[0]
    selection = partial_preprocessing(img_test_array)
    
    area_total = lesion_region.area
    img_mask = lesion_region.image
    horizontal_flip = np.fliplr(img_mask)
    diff_horizontal = img_mask * ~horizontal_flip
    
    vertical_flip = np.flipud(img_mask)
    diff_vertical = img_mask * ~vertical_flip
    
    diff_horizontal_area = np.count_nonzero(diff_horizontal)
    diff_vertical_area = np.count_nonzero(diff_vertical)
    a1 = diff_horizontal_area / area_total # Asymmetry score across x-axis
    a2 = diff_vertical_area / area_total # Asymmetry score across y-axis
    asymm_idx = 0.5 * (a1 + a2) # Asymmetricity Index
    ecc = lesion_region.eccentricity
    
    # Border Irregularity: Compactness index
    compactness_index = (4 * math.pi * area_total)/(lesion_region.perimeter*lesion_region.perimeter) #formula confusion(cleck later)
    b1 =  lesion_region.area/lesion_region.perimeter # area to perimeter ratio
    
    # Diameter
    eq_diameter = lesion_region.equivalent_diameter
    D1 = math.sqrt((4*lesion_region.area)/math.pi)
    D2 = (lesion_region.axis_major_length+lesion_region.axis_minor_length)/2
    d1 = ((D1+D2)/2)*0.265 # Average diameter of the lesion (1px = 0.265mm)
    d2 = lesion_region.axis_major_length-lesion_region.axis_minor_length # Difference between principal axes lengths
    
    # Color variegation:
    im = Image.fromarray(selection.astype('uint8'), 'RGB')
    stat = ImageStat.Stat(im)
    C_r = stat.stddev[0] / stat.extrema[0][1]
    C_g = stat.stddev[1] / stat.extrema[1][1]
    C_b = stat.stddev[2] / stat.extrema[2][1]

    # Texture: Gray Level Co-occurrence Matrix (GLCM)
    gray_img = cv2.cvtColor(img_test_array, cv2.COLOR_BGR2GRAY)
    glcm = feature.graycomatrix(image=img_as_ubyte(gray_img), distances=[1],angles=[0, np.pi/4, np.pi/2, np.pi * 3/2],symmetric=True, normed=True)
    correlation = np.mean(feature.graycoprops(glcm, prop='correlation'))
    homogeneity = np.mean(feature.graycoprops(glcm, prop='homogeneity'))
    energy = np.mean(feature.graycoprops(glcm, prop='energy'))
    contrast = np.mean(feature.graycoprops(glcm, prop='contrast'))

    properties.append(lesion_region.area)
    properties.append(lesion_region.perimeter)
    properties.append(diff_horizontal_area)
    properties.append(diff_vertical_area)
    properties.append(a1)
    properties.append(a2)
    properties.append(asymm_idx)
    properties.append(ecc)
    
    properties.append(compact_index)
    properties.append(b1)

    properties.append(d1)
    properties.append(d2)
    properties.append(eq_diameter * 0.265)
    
    properties.append(C_r)
    properties.append(C_g)
    properties.append(C_b)
    
    properties.append(correlation)
    properties.append(homogeneity)
    properties.append(energy)
    properties.append(contrast)
    
    colors_x = extcolors.extract_from_image(im, tolerance = 4, limit = 50)
    
    def hextriplet(colortuple):
        return '#' + ''.join(f'{i:02X}' for i in colortuple)

    def color_to_df(input):
        colors_pre_list = str(input).replace('([(','').split(', (')[0:-1]
        df_rgb = [i.split('), ')[0] + ')' for i in colors_pre_list]
        df_percent = [i.split('), ')[1].replace(')','') for i in colors_pre_list]
        #print(type(df_rgb[0]))
        #convert RGB to HEX code
    
        df_color_up = [hextriplet(tuple(map(int, ((test_str.replace('(','')).replace(')','')).split(', ')))) for test_str in df_rgb]
    
        df = pd.DataFrame(zip(df_color_up,df_rgb,df_percent), columns = ['c_code','rgb','occurence'])
        return df

    df_color = color_to_df(colors_x)

    list_color = list(df_color['c_code'])
    list_occurrence = [int(i) for i in list(df_color['occurence'])]
    list_rgb = [tuple(map(int, ((test_str.replace('(','')).replace(')','')).split(', '))) for test_str in list(df_color['rgb'])] 
    list_percent = [round((p*100)/sum(list_occurrence),3) for p in list_occurrence]

    color = [0]*6
    for i in range(0,len(list_rgb)):
        r = list_rgb[i][0]
        g = list_rgb[i][1]
        b = list_rgb[i][2]
        p = list_percent[i]
    
        if r>=(0.8*255) and g>=(0.8*255) and b>=(0.8*255):
            color[0]+=p #white
        elif r>=(0.588*255) and g<(0.2*255) and b<(0.2*255):
            color[1]+=p #red
        elif (r>=(0.588*255) and r<=(0.94*255)) and (g>(0.2*255) and g<=(0.588*255)) and (b>0 and b<(0.392*255)):
            color[2]+=p #lightbrown
        elif (r>(0.243*255) and r<(0.56*255)) and g<(0.392*255) and (b>0 and b<(0.392*255)):
            color[3]+=p #darkbrown
        elif r<=(0.588*255) and (g>=(0.392*255) and g<=(0.588*255)) and (b>=(0.490*255) and b<=(0.588*255)):
            color[4]+=p #bluegray
        elif r<=(0.243*255) and g<=(0.243*255) and b<=(0.243*255):
            color[5]+=p #black
    
    # using list comprehension
    # to get numbers > 5%
    color_count = len([i for i in color if i > 5])    
    properties.append(color_count)
    
    #properties list:
    #[area, perimeter, dLx, dLy, a1, a2, AI, eccentricity, compactness_index, b1, d1, d2, equivalent_diameter, C_r, C_g, C_b, correlation, homogeneity, energy, contrast, color_count]

    return properties

In [ ]:
print(len(img_test_array))
print(len(feature_extraction(img_test_array)))
print(feature_extraction(img_test_array))

In [ ]:
df_train = pd.DataFrame(columns = ['area', 'perimeter', 'dLx', 'dLy', 'a1', 'a2', 'AI', 'eccentricity', 'compactness_index', 'b1', 'd1', 'd2', 'equivalent_diameter', 'C_r', 'C_g', 'C_b', 'correlation', 'homogeneity', 'energy', 'contrast', 'color_count'])
df_test = pd.DataFrame(columns = ['area', 'perimeter', 'dLx', 'dLy', 'a1', 'a2', 'AI', 'eccentricity', 'compactness_index', 'b1', 'd1', 'd2', 'equivalent_diameter', 'C_r', 'C_g', 'C_b', 'correlation', 'homogeneity', 'energy', 'contrast', 'color_count'])

In [ ]:
print(type(X_train[0]))
print(type(img_test_array))
print(X_train[0].shape)
print(img_test_array.shape)

In [ ]:
p = feature_extraction(X_test[5])
print(p)
print(len(p))
print(type(p))

In [ ]:
'''
tdata = list()
for i in range(X_train.shape[0]):
    p = feature_extraction(X_train[i])
    tdata.append(p)
    print(i, end = " ")
df_train = pd.DataFrame(tdata, columns = ['area', 'perimeter', 'dLx', 'dLy', 'a1', 'a2', 'AI', 'eccentricity', 'compactness_index', 'b1', 'd1', 'd2', 'equivalent_diameter', 'C_r', 'C_g', 'C_b', 'correlation', 'homogeneity', 'energy', 'contrast', 'color_count'])
'''

In [ ]:
#df_train.head()

In [ ]:
# add classes
#df_train['dianosis'] = y_train

In [ ]:
#df_train.head()

In [ ]:
#df_train.to_csv('df_train.csv')

In [ ]:
tdata = list()
for i in range(X_test.shape[0]):
    p = feature_extraction(X_test[i])
    tdata.append(p)
    print(i, end = " ")
df_test = pd.DataFrame(tdata, columns = ['area', 'perimeter', 'dLx', 'dLy', 'a1', 'a2', 'AI', 'eccentricity', 'compactness_index', 'b1', 'd1', 'd2', 'equivalent_diameter', 'C_r', 'C_g', 'C_b', 'correlation', 'homogeneity', 'energy', 'contrast', 'color_count'])

In [ ]:
df_test.head()

In [ ]:
# add classes
df_test['dianosis'] = y_test

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv('df_test.csv')

In [ ]:
# Display first 15 images of moles, and how they are classified
w=40
h=30
fig=plt.figure(figsize=(12, 8))
columns = 5
rows = 3

for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    if y_train[i] == 0:
        ax.title.set_text('Benign')
    else:
        ax.title.set_text('Malignant')
    plt.imshow(X_train[i], interpolation='nearest')
plt.savefig('image_classes.png', dpi = 300)
plt.show()

In [ ]:
# Display first 15 images of moles, and how they are classified (POST Pre-Processing)
w=40
h=30
fig=plt.figure(figsize=(12, 8))
columns = 5
rows = 3

for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    if y_train[i] == 0:
        ax.title.set_text('Benign')
    else:
        ax.title.set_text('Malignant')
    plt.imshow(preprocessing(X_train[i]), cmap="gray")
plt.savefig('preprocessed_image_classes.png', dpi = 300)
plt.show()

In [ ]:
'''
#print(X_train.shape[0])
t = list()
for i in range(X_train.shape[0]):
    t.append(partial_preprocessing(X_train[i]))
    print(i, end = " ")
X_train = t
'''

In [ ]:
#X_train = np.array(X_train)
#print(X_train.shape)

In [ ]:
#np.save("x_train", X_train)

In [ ]:
#print(X_test.shape[0])
t = list()
for i in range(X_test.shape[0]):
    t.append(partial_preprocessing(X_test[i]))
    print(i, end = " ")
X_test = t

In [ ]:
X_test = np.array(X_test)
print(X_test.shape)

In [ ]:
np.save("x_test", X_test)

In [ ]:
#np.save("y_train", y_train)
np.save("y_test", y_test)

In [ ]:
y_train = np.load('/kaggle/input/skincancer-keras-cnn/iisic_train_test_segmented_image_array/y_train.npy')
y_test = np.load('/kaggle/input/skincancer-keras-cnn/iisic_train_test_segmented_image_array/y_test.npy')
X_train = np.load('/kaggle/input/skincancer-keras-cnn/iisic_train_test_segmented_image_array/x_train.npyy')
X_test = np.load('/kaggle/input/skincancer-keras-cnn/iisic_train_test_segmented_image_array/x_test.npy')

In [ ]:
plt.imshow(X_train[0])

**Step 4 : Normalization:** Normalize all Values of the pictures by dividing all the RGB values by 255

In [ ]:
# With data augmentation to prevent overfitting 
X_train = X_train/255.
X_test = X_test/255.

In [ ]:
print(X_train.shape)

In [ ]:
#Splitting training into Train and Validatation sets
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size = 0.2,random_state=123)

In [ ]:
print(y_train)

# **Feature Training**

In [ ]:
df_test = pd.read_csv('/kaggle/input/skincancer-keras-cnn/SkinCancer Features/df_test.csv')
df_train = pd.read_csv('/kaggle/input/skincancer-keras-cnn/SkinCancer Features/df_train.csv')

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
# Normalize the sensor response columns using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train[['area', 'perimeter', 'dLx', 'dLy', 'a1', 'a2', 'AI', 'eccentricity', 'compactness_index', 'b1', 'd1', 'd2', 'equivalent_diameter', 'C_r', 'C_g', 'C_b', 'correlation', 'homogeneity', 'energy', 'contrast', 'color_count']] = scaler.fit_transform(df_train[['area', 'perimeter', 'dLx', 'dLy', 'a1', 'a2', 'AI', 'eccentricity', 'compactness_index', 'b1', 'd1', 'd2', 'equivalent_diameter', 'C_r', 'C_g', 'C_b', 'correlation', 'homogeneity', 'energy', 'contrast', 'color_count']])
# Remove rows with missing values
df_train.dropna(inplace=True)

df_test[['area', 'perimeter', 'dLx', 'dLy', 'a1', 'a2', 'AI', 'eccentricity', 'compactness_index', 'b1', 'd1', 'd2', 'equivalent_diameter', 'C_r', 'C_g', 'C_b', 'correlation', 'homogeneity', 'energy', 'contrast', 'color_count']] = scaler.fit_transform(df_test[['area', 'perimeter', 'dLx', 'dLy', 'a1', 'a2', 'AI', 'eccentricity', 'compactness_index', 'b1', 'd1', 'd2', 'equivalent_diameter', 'C_r', 'C_g', 'C_b', 'correlation', 'homogeneity', 'energy', 'contrast', 'color_count']])
# Remove rows with missing values
df_test.dropna(inplace=True)

In [ ]:
df_train = df_train.iloc[:,1:] #remove Unnamed: 0
df_test = df_test.iloc[:,1:]

In [ ]:
df_train.columns

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
# Visualize the distribution of dignosis classes
plt.figure(figsize=(7,7))
sns.countplot(data=df_train, x='dianosis', palette = "Set2")
plt.title('Distribution of Diagnostic Classes')
plt.savefig('distribution.png', dpi = 300)

In [ ]:
# finding the correlation matrix
correlation_matrix = df_train.corr()
correlation_matrix.head()

In [ ]:
# Explore the correlation between Diagnosis Classes and other features
plt.figure(figsize=(8,7))
sns.heatmap(correlation_matrix, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.savefig('coorelation.png', dpi = 300)
plt.show()

In [ ]:
# defining the threshold

# get the correlation of target column (ProtocolName) with the rest of the columns
sorted_corr_matrix_protocolName = correlation_matrix['dianosis'].sort_values(ascending=False)
allKeys = sorted_corr_matrix_protocolName.keys()

pos_value_columns = []
# printing the positive correlation value
for colName in allKeys:
    print(sorted_corr_matrix_protocolName[colName], colName)
    if (sorted_corr_matrix_protocolName[colName] > 0 and colName != 'dianosis'):
        pos_value_columns.append(colName)
print(pos_value_columns)
print(len(pos_value_columns))

In [ ]:
X_train=df_train[pos_value_columns]     #X-input features
y_train=df_train['dianosis']

X_test=df_test[pos_value_columns]     #X-input features
y_test=df_test['dianosis']

In [ ]:
X_train

In [ ]:
# Define estimator for feature selection
estimator = RandomForestClassifier(n_estimators=100, random_state=42)
# KNeighborsClassifier(n_neighbors=21, weights='distance', metric='manhattan')
# Define recursive feature elimination with cross-validation
rfecv = RFECV(estimator=estimator, step=1, cv=5, scoring='accuracy')

# Fit RFECV to training data
rfecv.fit(X_train, y_train)

# Print selected features
print("Selected Features: ", X_train.columns[rfecv.support_])

# ['d1', 'dLy', 'dLx', 'area', 'contrast', 'C_g', 'perimeter', 'C_b', 'C_r']

In [ ]:
# Extra Trees Classifier

from sklearn.ensemble import ExtraTreesClassifier

# Building the model
extra_tree_forest = ExtraTreesClassifier(n_estimators = 50,criterion ='gini', max_features = 'log2')
  
# Training the model
extra_tree_forest.fit(X_train, y_train)
  
# Computing the importance of each feature
feature_importance = extra_tree_forest.feature_importances_
  
# Normalizing the individual importances
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

# find out the best ones again and keep it here

In [ ]:
threshold = 0.01
# split it up
above_threshold = np.maximum(feature_importance_normalized  - threshold, 0)
below_threshold = np.minimum(feature_importance_normalized , threshold)

# and plot it
fig, ax = plt.subplots(figsize = (20,5))
ax.tick_params(labelrotation=90)
ax.set_xlabel('Feature Labels')
ax.set_ylabel('Feature Importances')
ax.set_title('Comparison of different Feature Importances')
ax.bar(X_train.columns, below_threshold)
ax.bar(X_train.columns, above_threshold, color="r",bottom=below_threshold)
fig.savefig('extratrees.png', dpi = 300)

In [ ]:
selected_features = []
for i in range(len(X_train.columns)):
    if below_threshold[i] == 0.01:
        selected_features.append(X_train.columns[i])
selected_features

# ['d1','dLy','dLx','equivalent_diameter','area','contrast','C_g','perimeter','C_b','C_r','AI']

In [ ]:
rfecv = list(X_train.columns[rfecv.support_])

# Python program to illustrate the intersection of two lists in most simple way
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
 
features = intersection(selected_features,rfecv)
features

# ['d1', 'dLy', 'dLx', 'area', 'contrast', 'C_g', 'perimeter', 'C_b', 'C_r']

In [ ]:
features = ['d1', 'dLy', 'dLx', 'area', 'contrast', 'C_g', 'perimeter', 'C_b', 'C_r']

In [ ]:
Xf_train = df_train[features]
yf_train = df_train['dianosis']

Xf_test = df_test[features]
yf_test = df_test['dianosis']

In [ ]:
Xf_train

In [ ]:
#Use heatmap to see corelation between variables
plt.figure(figsize = (12,10))
sns.heatmap(Xf_train.corr(),annot=True,cmap='viridis')
plt.title('Heatmap of co-relation between variables',fontsize=2)
plt.savefig('coorelation_heatmap.png', dpi = 300)
plt.show()

### **Models**

In [ ]:
# use GridSearchCV to tune the hyperparameters for the meta-learners


# Define the models to evaluate
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Machines': SVC(random_state=42),
    'K-Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Cat Boost': CatBoostClassifier(random_state=42),
    'XGB': XGBClassifier(random_state=42)
}
    
# Define the hyperparameters to tune for each model
params = {
    'Logistic Regression': {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'solver': ['newton-cg', 'lbfgs', 'liblinear']},
    'Random Forest': {'n_estimators': [10, 50, 100, 250, 500], 'max_depth': [5, 10, 20]},
    'Gradient Boosting': {'n_estimators': [10, 50, 100, 250, 500], 'learning_rate': [0.0005, 0.0001, 0.005, 0.001, 0.01, 0.1]},
    'Support Vector Machines': {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel': ['linear', 'rbf']},
    'K-Neighbors': { 'n_neighbors': [3, 5, 7, 11, 21], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']},
    'Decision Tree': {'max_features': ['auto'],'ccp_alpha': [0.1, .01, .001],'max_depth' : [5, 10, 20],'criterion' :['gini', 'entropy']},
    'Cat Boost': {'iterations': [50,500,5000], 'max_depth': [5, 10, 20], 'loss_function': ['Logloss', 'CrossEntropy', 'MultiClass', 'MultiClassOneVsAll'], 'learning_rate': [0.0005, 0.0001, 0.005, 0.001, 0.01, 0.1], 'eval_metric': ['AUC']},
    'XGB': {'max_depth': [5, 10, 20], 'n_estimators': [10, 50, 100, 250, 500], 'learning_rate': [0.0005, 0.0001, 0.005, 0.001, 0.01, 0.1]}  
}
    
# Create a list to store the results of each model
results = []
    
# Loop through each model and perform GridSearchCV
for name, model in models.items():
    clf = RandomizedSearchCV(model, params[name], cv=3, scoring='accuracy', n_jobs = -1)
    # fit the meta learner
    clf.fit(Xf_train, yf_train)
    
# Add the model name and best accuracy score to the results list
results.append({'model': name, 'best_score': clf.best_score_, 'best_params': clf.best_params_})

# Print the results for each model
for result in results:
    print(f"{result['model']}: Best score = {result['best_score']:.4f}, Best params = {result['best_params']}")
    
# K-Nearest: Best score = 0.7478, Best params = {'metric': 'manhattan', 'n_neighbors': 21, 'weights': 'distance'}
# XGB: Best score = 0.7774, Best params = {'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.1}

In [ ]:

# xgb = XGBClassifier(random_state=42, n_estimators=10, max_depth=20, learning_rate=0.0005)
xgb = XGBClassifier(random_state=42, n_estimators=500, max_depth=5, learning_rate=0.1)
xgb.fit(Xf_train,yf_train)

# save the model to disk
filename = 'xgb_model.sav'
pickle.dump(xgb, open(filename, 'wb'))

y_pred_xgb = xgb.predict(Xf_test)
print(metrics.accuracy_score(yf_test, y_pred_xgb))

### **Individual Tunining**

In [ ]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'solver': ['newton-cg', 'lbfgs', 'liblinear']}
lr = RandomizedSearchCV(LogisticRegression(random_state=42), params, cv = 3, n_jobs=-1)
lr.fit(Xf_train, yf_train)
print(lr.best_score_)
print(lr.best_params_)

ypred = lr.predict(Xf_test)
print("Testing: ",metrics.accuracy_score(yf_test, ypred))

# save the model to disk
filename = 'lr_model.sav'
pickle.dump(lr, open(filename, 'wb'))


# 0.7755024649222602 {'solver': 'liblinear', 'C': 1000}
# Testing:  0.7287878787878788

In [ ]:
params = {'n_estimators': [10, 50, 100, 250, 500], 'max_depth': [5, 10, 20]}
rf = RandomizedSearchCV(RandomForestClassifier(random_state=42), params, cv = 3, n_jobs=-1)
rf.fit(Xf_train, yf_train)
print(rf.best_score_)
print(rf.best_params_)

ypred = rf.predict(Xf_test)
print("Testing: ",metrics.accuracy_score(yf_test, ypred))

# save the model to disk
filename = 'rf_model.sav'
pickle.dump(rf, open(filename, 'wb'))

# 0.7682973075464542 {'n_estimators': 500, 'max_depth': 20}
# Testing:  0.7560606060606061

In [ ]:
params =  {'n_neighbors': [3, 5, 7, 11, 21], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}
knn = RandomizedSearchCV(KNeighborsClassifier(), params, cv = 3,  n_jobs=-1)
knn.fit(Xf_train, yf_train)
print(knn.best_score_)
print(knn.best_params_)

ypred = knn.predict(Xf_test)
print("Testing: ",metrics.accuracy_score(yf_test, ypred))

# save the model to disk
filename = 'knn_model.sav'
pickle.dump(knn, open(filename, 'wb'))

# 0.7398558968524839 {'weights': 'distance', 'n_neighbors': 11, 'metric': 'manhattan'}
# Testing:  0.7545454545454545

In [ ]:
params = {'max_features': ['auto'],'ccp_alpha': [0.1, .01, .001],'max_depth' : [5, 10, 20],'criterion' :['gini', 'entropy']},
dt = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, cv = 3, n_jobs=-1)
dt.fit(Xf_train, yf_train)
print(dt.best_score_)
print(dt.best_params_)

ypred = dt.predict(Xf_test)
print("Testing: ",metrics.accuracy_score(yf_test, ypred))

# save the model to disk
filename = 'dt_model.sav'
pickle.dump(dt, open(filename, 'wb'))

# 0.7171027682973076 {'max_features': 'auto', 'max_depth': 5, 'criterion': 'entropy', 'ccp_alpha': 0.001}
# Testing:  0.7106060606060606

In [ ]:
params = {'iterations': [50,500,5000], 'max_depth': [5, 10, 20], 'loss_function': ['Logloss', 'CrossEntropy', 'MultiClass', 'MultiClassOneVsAll'], 'learning_rate': [0.0005, 0.0001, 0.005, 0.001, 0.01, 0.1], 'eval_metric': ['AUC']}
cat = RandomizedSearchCV(CatBoostClassifier(random_state=42), params, cv = 3, n_jobs=-1)
cat.fit(Xf_train, yf_train)
print(cat.best_score_)
print(cat.best_params_)

ypred = cat.predict(Xf_test)
print("Testing: ",metrics.accuracy_score(yf_test, ypred))

# save the model to disk
filename = 'cat_model.sav'
pickle.dump(cat, open(filename, 'wb'))

# 0.7808115282518013 {'max_depth': 5, 'loss_function': 'Logloss', 'learning_rate': 0.1, 'iterations': 500, 'eval_metric': 'AUC'}
# Testing:  0.7621212121212121

In [ ]:
params = {'max_depth': [5, 10, 20], 'n_estimators': [10, 50, 100, 250, 500], 'learning_rate': [0.0005, 0.0001, 0.005, 0.001, 0.01, 0.1]}
xgb = RandomizedSearchCV(XGBClassifier(random_state=42), params, cv = 3, n_jobs=-1)
xgb.fit(Xf_train, yf_train)
print(xgb.best_score_)
print(xgb.best_params_)

ypred = xgb.predict(Xf_test)
print("Testing: ",metrics.accuracy_score(yf_test, ypred))

# save the model to disk
filename = 'xgb_model.sav'
pickle.dump(xgb, open(filename, 'wb'))

# 0.7849829351535836 {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.1}
# Testing:  0.753030303030303

In [ ]:
params = {'n_estimators': [10, 50, 100, 250, 500], 'learning_rate': [0.0005, 0.0001, 0.005, 0.001, 0.01, 0.1]}
gbc = RandomizedSearchCV(GradientBoostingClassifier(random_state=42), params, cv = 3, n_jobs=-1)
gbc.fit(Xf_train, yf_train)
print(gbc.best_score_)
print(gbc.best_params_)

ypred = gbc.predict(Xf_test)
print("Testing: ",metrics.accuracy_score(yf_test, ypred))

# save the model to disk
filename = 'gbc_model.sav'
pickle.dump(gbc, open(filename, 'wb'))

# 0.7796738718240425 {'n_estimators': 250, 'learning_rate': 0.1}
# Testing:  0.7575757575757576

In [ ]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel': ['linear', 'rbf']}
svc = RandomizedSearchCV(SVC(random_state=42), params, cv = 3, n_jobs=-1)
svc.fit(Xf_train, yf_train)
print(svc.best_score_)
print(svc.best_params_)

ypred = svc.predict(Xf_test)
print("Testing: ",metrics.accuracy_score(yf_test, ypred))

# save the model to disk 
filename = 'svc_model.sav'
pickle.dump(svc, open(filename, 'wb'))

# 0.7982555934774366 {'kernel': 'rbf', 'C': 100}
# Testing:  0.7545454545454545

In [ ]:
# Confusion Matrix
confusion_matrix = metrics.confusion_matrix(yf_test, y_pred_xgb)

plt.figure(figsize=(8,6))
fx=sns.heatmap(confusion_matrix, annot=True, fmt=".2f",cmap="GnBu")
fx.set_title('Confusion Matrix \n');
fx.set_xlabel('\n Predicted Values\n')
fx.set_ylabel('Actual Values\n')
labels = ['Benign','Malignant']
fx.xaxis.set_ticklabels(labels, rotation=90)
fx.yaxis.set_ticklabels(labels, rotation=0)
#plt.savefig('confusionmatrix.png', dpi = 300)
plt.show()

In [ ]:
# Classification Report

print(metrics.classification_report(yf_test, y_pred_xgb))

In [ ]:
# Classification Report

print(metrics.classification_report(yf_test, y_pred_xgb))

#accuracy on test set
result = xgb.score(Xf_test, yf_test)
print("Accuracy - test set: %.2f%%" % (result*100.0))

#precision on test set
precision = precision_score(yf_test, y_pred_xgb)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(yf_test, y_pred_xgb)
print('Recall: %f' % recall)

# f1: tp / (tp + fp + fn)
f1 = f1_score(yf_test, y_pred_xgb)
print('F1 score: %f' % f1)

# ROC curve
# predict probabilities
probs = xgb.predict_proba(Xf_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]

auc = roc_auc_score(yf_test, probs)
print('AUC - Test Set: %.2f%%' % (auc*100))

# calculate roc curve
fpr, tpr, thresholds = roc_curve(yf_test, probs)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False positive rate')
plt.ylabel('Sensitivity/ Recall')
#plt.savefig('auc_roc.png', dpi=300)
# show the plot
plt.show()

### **RNN**

I used the Keras Sequential API, where you have just to add one layer at a time, starting from the input.

The first is the convolutional (Conv2D) layer. It is like a set of learnable filters. I choosed to set 64 filters for the two firsts conv2D layers. Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. The kernel filter matrix is applied on the whole image. Filters can be seen as a transformation of the image.

The CNN can isolate features that are useful everywhere from these transformed images (feature maps).

The second important layer in CNN is the pooling (MaxPool2D) layer. This layer simply acts as a downsampling filter. It looks at the 2 neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting. We have to choose the pooling size (i.e the area size pooled each time) more the pooling dimension is high, more the downsampling is important.

Combining convolutional and pooling layers, CNN are able to combine local features and learn more global features of the image.

Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored (setting their wieghts to zero) for each training sample. This drops randomly a propotion of the network and forces the network to learn features in a distributed way. This technique also improves generalization and reduces the overfitting.

'relu' is the rectifier (activation function max(0,x). The rectifier activation function is used to add non linearity to the network.

The Flatten layer is use to convert the final feature maps into a one single 1D vector. This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous convolutional layers.

In the end i used the features in one fully-connected (Dense) layer which is just artificial an neural networks (ANN) classifier.

In [ ]:
Xf_train.shape

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras import regularizers

# Define the model architecture
def create_model(neurons, dropout_rate, kernel_regularizer, learning_rate):
    input_shape = (Xf_train.shape[1],)
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons//2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons//4, activation='relu', kernel_regularizer=regularizers.l2(kernel_regularizer)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons//8, activation='relu', kernel_regularizer=regularizers.l2(kernel_regularizer)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons//16, activation='relu', kernel_regularizer=regularizers.l2(kernel_regularizer)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons/32, activation='relu', kernel_regularizer=regularizers.l2(kernel_regularizer)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

# Create the KerasClassifier wrapper for scikit-learn
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters search space
neurons = [64, 128, 256, 512, 1024]
dropout_rate = [0, 0.25, 0.5, 0.75]
kernel_regularizer = [0.01, 0.001, 0.0001]
learning_rate = [0.0005, 0.0001, 0.005, 0.001, 0.01, 0.1]
batch_size = [16, 32, 64]
epochs = [50, 100, 150, 300]

param_grid = dict(neurons=neurons, dropout_rate=dropout_rate, kernel_regularizer=kernel_regularizer, learning_rate=learning_rate, batch_size=batch_size, epochs=epochs)

# Perform the randomized search with cross-validation
n_iter_search = 50
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=n_iter_search, cv=3, n_jobs = -1)
random_search.fit(Xf_train, yf_train, validation_data = (Xf_test, yf_test))

# Print the best parameters and score
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras import regularizers

# Define the parameters

neurons = 1024
dropout_rate = 0.25
kernel_regularizer = 0.001
learning_rate = 0.00001
batch_size = 16
epochs = 300

input_shape = (Xf_train.shape[1],)
model = Sequential()
model.add(Dense(neurons, activation='relu', input_shape=input_shape))
model.add(Dropout(dropout_rate))
model.add(Dense(neurons//2, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(neurons//4, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(neurons//8, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(neurons//16, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(neurons/32, activation='relu', kernel_regularizer=regularizers.l2(kernel_regularizer)))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))

opt = Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'],)


history = model.fit(Xf_train, yf_train, validation_data = (Xf_test, yf_test), epochs=epochs, batch_size=batch_size) #, callbacks=[tf.keras.callbacks.ReduceLROnPlateau()])

In [ ]:
# You can also plot the model architecture using `plot_model`
plot_model(model, show_shapes=True)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model accuracy.png', dpi = 300)
plt.show()

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('model accuracy vs loss')
plt.xlabel('epoch')
plt.legend(['accuracy', 'loss'], loc='upper left')
plt.savefig('model accuracy vs loss.png', dpi = 300)
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model loss.png', dpi = 300)
plt.show()

# summarize history for accuracy
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['val_loss'])
plt.title('model validation accuracy vs loss')
plt.xlabel('epoch')
plt.legend(['val_accuracy', 'val_loss'], loc='upper left')
plt.savefig('model validation accuracy.png', dpi = 300)
plt.show()

In [ ]:
#model.save('featureRNN.h5')

In [ ]:
model = load_model('/kaggle/input/skincancer-keras-cnn/featureRNN.h5')

In [ ]:
y_pred_proba = model.predict(Xf_test)
# Apply a threshold of 0.5 to obtain the predicted class labels
threshold = 0.5
y_pred = (y_pred_proba > threshold).astype(int).flatten()

In [ ]:
# Classification Report

print(metrics.classification_report(yf_test, y_pred))

In [ ]:
# Visualize the distribution of RH_type classes
df_y_pred = pd.DataFrame(y_pred, columns=['Predicted Diagnosis'])
df_y_test = pd.DataFrame(list(yf_test), columns=['Actual Diagnosis'])

fig, ax =plt.subplots(1,2, figsize=(15,5))
sns.countplot (x = df_y_test['Actual Diagnosis'], palette = "Set2", ax=ax[0])
sns.countplot (x = df_y_pred['Predicted Diagnosis'], palette = "Set2", ax=ax[1])
fig.savefig('predicted diagnosis.png', dpi = 300)
fig.show()

In [ ]:
# Confusion Matrix
confusion_matrix = metrics.confusion_matrix(yf_test, y_pred)

plt.figure(figsize=(8,6))
fx=sns.heatmap(confusion_matrix, annot=True, fmt=".2f",cmap="GnBu")
fx.set_title('Confusion Matrix \n');
fx.set_xlabel('\n Predicted Values\n')
fx.set_ylabel('Actual Values\n')
labels = ['Benign','Malignant']
fx.xaxis.set_ticklabels(labels, rotation=90)
fx.yaxis.set_ticklabels(labels, rotation=0)
plt.savefig('confusionmatrix.png', dpi = 300)
plt.show()

In [ ]:
# Classification Report

print(metrics.classification_report(yf_test, y_pred))

#accuracy on test set
result = xgb.score(Xf_test, yf_test)
print("Accuracy - test set: %.2f%%" % (result*100.0))

#precision on test set
precision = precision_score(yf_test, y_pred)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(yf_test, y_pred)
print('Recall: %f' % recall)

# f1: tp / (tp + fp + fn)
f1 = f1_score(yf_test, y_pred)
print('F1 score: %f' % f1)

# ROC curve
# predict probabilities
probs = xgb.predict_proba(Xf_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]

auc = roc_auc_score(yf_test, probs)
print('AUC - Test Set: %.2f%%' % (auc*100))

# calculate roc curve
fpr, tpr, thresholds = roc_curve(yf_test, probs)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False positive rate')
plt.ylabel('Sensitivity/ Recall')
plt.savefig('auc_roc.png', dpi=300)
# show the plot
plt.show()

# **Stacked Ensemble**

In [ ]:
rf = pickle.load(open('/kaggle/input/skincancer-keras-cnn/Features_sklearn_models/rf_model.sav', 'rb'))
kn = pickle.load(open('/kaggle/input/skincancer-keras-cnn/Features_sklearn_models/knn_model.sav', 'rb'))
xgb = pickle.load(open('/kaggle/input/skincancer-keras-cnn/Features_sklearn_models/xgb_model.sav', 'rb'))
gbc = pickle.load(open('/kaggle/input/skincancer-keras-cnn/Features_sklearn_models/gbc_model.sav', 'rb'))
cat = pickle.load(open('/kaggle/input/skincancer-keras-cnn/Features_sklearn_models/cat_model.sav', 'rb'))
svc = pickle.load(open('/kaggle/input/skincancer-keras-cnn/Features_sklearn_models/svc_model.sav', 'rb'))
rnn = load_model('/kaggle/input/skincancer-keras-cnn/featureRNN.h5')

In [ ]:
y_pred_rf = rf.predict(Xf_test)
y_pred_kn = kn.predict(Xf_test)
y_pred_cat = cat.predict(Xf_test)
y_pred_xgb = xgb.predict(Xf_test)
y_pred_gbc = gbc.predict(Xf_test)
y_pred_svc = svc.predict(Xf_test)

In [ ]:
#y_pred = list(rnn.predict(Xf_test))
y_pred = list(rnn.predict(Xf_test))
print(type(y_pred))
y_pred_rnn = []
for i in range(len(y_pred)):
    a = list(y_pred[i])
    maxpos = a.index(max(a))
    y_pred_rnn.append(maxpos)

y_pred_rnn = np.array(y_pred_rnn)
y_pred_rnn.shape

In [ ]:
#y_pred_cat = y_pred_cat.reshape(y_pred_cat.shape[0],)

In [ ]:
print(y_pred_rf.shape)
print(y_pred_kn.shape)
print(y_pred_xgb.shape)
print(y_pred_gbc.shape)
print(y_pred_cat.shape)
print(y_pred_svc.shape)
print(y_pred_rnn.shape)

In [ ]:
combined_y_pred = np.stack((y_pred_rf, y_pred_kn, y_pred_xgb, y_pred_gbc,y_pred_cat, y_pred_svc, y_pred_rnn), axis=1)
combined_y_pred.shape

In [ ]:
stackedX = combined_y_pred

In [ ]:
stackedX.shape

In [ ]:
yf_test.shape

In [ ]:
# use GridSearchCV to tune the hyperparameters for the meta-learners

def fit_stacked_model(stackedX, inputX, inputy):
    # create dataset using ensemble    
    # create dataset using ensemble'
    # stackedX = stacked_dataset(members, inputX)
    # Define the models to evaluate
    models = {        
        'Logistic Regression': LogisticRegression(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'K-Nearest': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Cat Boost': CatBoostClassifier(random_state=42),
        'Support Vector Machines': SVC(random_state=42),
        'XGB': XGBClassifier(random_state=42) 
    }
    
    # Define the hyperparameters to tune for each model
    params = {
        'Logistic Regression': {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'solver': ['newton-cg', 'lbfgs', 'liblinear']},
        'Random Forest': {'n_estimators': [10, 50, 100, 250, 500], 'max_depth': [5, 10, 20]},
        'Gradient Boosting': {'n_estimators': [10, 50, 100, 250, 500], 'learning_rate': [[ 0.001, 0.005, 0.0001, 0.0005]]},
        'K-Nearest': {'n_neighbors': [3, 5, 7, 11, 21], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']},
        'Decision Tree': {'max_features': ['auto'],'ccp_alpha': [0.1, .01, .001],'max_depth' : [5, 10, 20],'criterion' :['gini', 'entropy']},
        'Cat Boost': {'iterations': [50,500,5000], 'max_depth': [5, 10, 20], 'loss_function': ['MultiClass'],  'learning_rate': [ 0.001, 0.005, 0.0001, 0.0005], 'eval_metric': ['MultiClass']},
        'Support Vector Machines': {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel': ['linear', 'rbf']},
        'XGB': {'max_depth': [5, 10, 20], 'n_estimators': [10, 50, 100, 250, 500], 'learning_rate': [0.0005, 0.0001, 0.005, 0.001, 0.01, 0.1]}  
    }
    
    # Create a list to store the results of each model
    results = []
    
    # Loop through each model and perform GridSearchCV
    for name, model in models.items():
        clf = RandomizedSearchCV(model, params[name], cv=5,  n_jobs=-1, scoring='accuracy')
        # fit the meta learner
        clf.fit(stackedX, inputy)
    
    # Add the model name and best accuracy score to the results list
    results.append({'model': name, 'best_score': clf.best_score_, 'best_params': clf.best_params_})

    # Print the results for each model
    for result in results:
        print(f"{result['model']}: Best score = {result['best_score']:.4f}, Best params = {result['best_params']}")
    
    return clf

model = fit_stacked_model(stackedX, Xf_test,yf_test)

In [ ]:
# stacked meta learner

model = XGBClassifier(n_estimators=500, max_depth=5, learning_rate=0.1)
model.fit(stackedX,yf_test)

In [ ]:
# make a prediction with the stacked model
def stacked_prediction(model, stackedX):
    y_pred = model.predict(stackedX)
    return y_pred

# evaluate model on test set -
y_pred = stacked_prediction(model, stackedX)

In [ ]:
# Classification Report

print(metrics.classification_report(yf_test, y_pred))

In [ ]:
# Confusion Matrix
confusion_matrix = metrics.confusion_matrix(yf_test, y_pred)

plt.figure(figsize=(8,6))
fx=sns.heatmap(confusion_matrix, annot=True, fmt=".2f",cmap="GnBu")
fx.set_title('Confusion Matrix \n');
fx.set_xlabel('\n Predicted Values\n')
fx.set_ylabel('Actual Values\n')
#labels = ['Audio-Streaming','Browsing','Chat','Email','File-Transfer','P2P', 'Video-Streaming','VOIP']
labels = ['Benign', 'Malignant']
fx.xaxis.set_ticklabels(labels, rotation=90)
fx.yaxis.set_ticklabels(labels, rotation=0)
plt.savefig('confusionmatrix.png', dpi = 300)
plt.show()

In [ ]:
#accuracy on test set
esult = model.score(stackedX, yf_test)
print("Accuracy - test set: %.2f%%" % (result*100.0))

#precision on test set
precision = precision_score(yf_test, y_pred)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(yf_test, y_pred)
print('Recall: %f' % recall)

# f1: tp / (tp + fp + fn)
f1 = f1_score(yf_test, y_pred)
print('F1 score: %f' % f1)

# ROC curve
# predict probabilities
probs = xgb.predict_proba(Xf_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]

auc = roc_auc_score(yf_test, probs)
print('AUC - Test Set: %.2f%%' % (auc*100))

# calculate roc curve
fpr, tpr, thresholds = roc_curve(yf_test, probs)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False positive rate')
plt.ylabel('Sensitivity/ Recall')
#plt.savefig('auc_roc.png', dpi=300)
# show the plot
plt.show()

In [ ]:
Xf_test